In [13]:
import xarray as xr
import numpy as np
import sys
import time
from scipy.stats import weibull_min
import dask.distributed as dd

root_dir = '/media/harish/SSD_4TB/EU_SCORES_project'
scripts_dir = f'{root_dir}/scripts'
sys.path.append(scripts_dir)

from data_processing.libraries import mean_statistics, std_statistics, wind_power_density

In [12]:
client.close()
cluster.close()

In [14]:
import dask.distributed as dd
cluster = dd.LocalCluster(n_workers=8, dashboard_address=':22722')
client = dd.Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:22722/status,
Dashboard: http://127.0.0.1:22722/status,Workers: 8
Total threads: 96,Total memory: 187.55 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39319,Workers: 8
Dashboard: http://127.0.0.1:22722/status,Total threads: 96
Started: Just now,Total memory: 187.55 GiB
Comm: tcp://127.0.0.1:34273,Total threads: 12
Dashboard: http://127.0.0.1:38123/status,Memory: 23.44 GiB
Nanny: tcp://127.0.0.1:41319,


In [24]:
root_dir = '/media/harish/SSD_4TB/EU_SCORES_project'
run = 'New_runs' #sys.argv[1]
case = 'Netherlands_coast' #sys.argv[2]
level = 80 #int(sys.argv[3])

run_dir=f'{root_dir}/WRFV4.4/EU_SCORES/{run}/{case}/Postprocessed/variablewise_files'
target_file = f'{run_dir}/wpd_{level}.nc'
print(run_dir,target_file)

rho = 1.225
chunks={"Time": 240,"south_north": -1,"west_east": -1}
ws = xr.open_dataset(f'{run_dir}/ws_{level}.nc',chunks=chunks)['ws']
ws

/media/harish/SSD_4TB/EU_SCORES_project/WRFV4.4/EU_SCORES/New_runs/Netherlands_coast/Postprocessed/variablewise_files /media/harish/SSD_4TB/EU_SCORES_project/WRFV4.4/EU_SCORES/New_runs/Netherlands_coast/Postprocessed/variablewise_files/wpd_80.nc


<xarray.DataArray 'ws' (Time: 271753, south_north: 199, west_east: 199)>
dask.array<open_dataset-0143c97b27e79fad92510bfc309bdccaws, shape=(271753, 199, 199), dtype=float32, chunksize=(240, 199, 199), chunktype=numpy.ndarray>
Coordinates:
    num_z_levels_stag  float32 ...
  * Time               (Time) datetime64[ns] 1990-01-01 ... 2021-01-01
    XLONG              (south_north, west_east) float32 dask.array<chunksize=(199, 199), meta=np.ndarray>
    XLAT               (south_north, west_east) float32 dask.array<chunksize=(199, 199), meta=np.ndarray>
Dimensions without coordinates: south_north, west_east

In [25]:
chunksize = 72000
# first chunk computation 
i = 0
chunk = ws.isel(Time=slice(i, i + chunksize)).compute()

In [26]:
wpd_chunk = wind_power_density(chunk,rho)
wpd_chunk

<xarray.DataArray 'wpd' (Time: 72000, south_north: 199, west_east: 199)>
array([[[3.66517029e+01, 3.71465721e+01, 3.72880707e+01, ...,
         2.78813076e+00, 2.78096819e+00, 2.82262135e+00],
        [3.73286476e+01, 3.75993271e+01, 3.72653503e+01, ...,
         2.42321420e+00, 2.55687928e+00, 2.78656912e+00],
        [3.80055237e+01, 3.81661911e+01, 3.78717232e+01, ...,
         2.09299827e+00, 2.35726476e+00, 2.69730759e+00],
        ...,
        [9.17764187e+00, 8.87759781e+00, 8.72315502e+00, ...,
         2.22945175e+01, 2.19789524e+01, 2.13598766e+01],
        [8.97626114e+00, 8.66027927e+00, 8.23601913e+00, ...,
         2.23270302e+01, 2.21600380e+01, 2.18874855e+01],
        [8.90608883e+00, 8.71552277e+00, 8.31313133e+00, ...,
         2.27599869e+01, 2.26152534e+01, 2.25195808e+01]],

       [[3.28274841e+01, 3.37390099e+01, 3.35361557e+01, ...,
         4.13882446e+00, 4.03982162e+00, 3.97023416e+00],
        [3.29559708e+01, 3.37410889e+01, 3.35049210e+01, ...,
         3.87182593e+00, 4.02825069e+00, 4.08077145e+00],
        [3.34550247e+01, 3.38309402e+01, 3.40489197e+01, ...,
         3.46152091e+00, 3.74790645e+00, 3.95554161e+00],
...
        [1.80252228e+01, 1.84176025e+01, 1.87012939e+01, ...,
         1.43264938e+02, 1.41958969e+02, 1.42569885e+02],
        [1.82420940e+01, 1.89487286e+01, 1.96387024e+01, ...,
         1.45285583e+02, 1.44102859e+02, 1.44401825e+02],
        [1.84114189e+01, 1.90581284e+01, 1.97735596e+01, ...,
         1.45567627e+02, 1.44755524e+02, 1.45053467e+02]],

       [[4.20949847e-01, 3.86710584e-01, 4.16285753e-01, ...,
         2.95226407e+00, 2.92204213e+00, 2.75941277e+00],
        [4.30554360e-01, 3.91236871e-01, 4.19499338e-01, ...,
         3.60895848e+00, 3.40997028e+00, 3.04278755e+00],
        [4.26540315e-01, 4.24490511e-01, 4.38643485e-01, ...,
         4.22083902e+00, 3.78032470e+00, 3.31649685e+00],
        ...,
        [3.68342438e+01, 3.71524277e+01, 3.74515915e+01, ...,
         1.72110397e+02, 1.71205322e+02, 1.71684433e+02],
        [3.71169243e+01, 3.78387794e+01, 3.87778893e+01, ...,
         1.73676575e+02, 1.73035416e+02, 1.73150436e+02],
        [3.73008728e+01, 3.79551277e+01, 3.89055367e+01, ...,
         1.73522263e+02, 1.73039047e+02, 1.73292267e+02]]], dtype=float32)
Coordinates:
    num_z_levels_stag  float32 80.0
  * Time               (Time) datetime64[ns] 1990-01-01 ... 1998-03-19T23:00:00
    XLONG              (south_north, west_east) float32 2.171 2.179 ... 3.642
    XLAT               (south_north, west_east) float32 50.94 50.94 ... 51.86
Dimensions without coordinates: south_north, west_east

In [27]:
wpd = wpd_chunk
del chunk, wpd_chunk

In [28]:
for i in range(chunksize, ws.Time.size, chunksize):
    start = time.time()
    chunk = ws.isel(Time=slice(i, i + chunksize)).compute()
    end = time.time()
    print(f'{i} to {i+chunksize} ws loading takes {end - start}s')

    start = time.time()
    wpd_chunk = wind_power_density(chunk,rho)
    end = time.time()
    print(f'{i} to {i+chunksize} chunk wpd computation takes {end - start}s')

    start = time.time()
    wpd = xr.concat([wpd, wpd_chunk], dim='Time')
    end = time.time()
    print(f'{i} to {i+chunksize} concatenate takes {end - start}s')
    
    del chunk, wpd_chunk

72000 to 144000 ws loading takes 85.80709528923035s
72000 to 144000 chunk wpd computation takes 106.11939263343811s
72000 to 144000 concatenate takes 85.68062734603882s
144000 to 216000 ws loading takes 73.68835878372192s
144000 to 216000 chunk wpd computation takes 139.74126863479614s
144000 to 216000 concatenate takes 172.50806665420532s
216000 to 288000 ws loading takes 34.78952074050903s
216000 to 288000 chunk wpd computation takes 48.91219353675842s
216000 to 288000 concatenate takes 207.8037645816803s


In [29]:
target_file

'/media/harish/SSD_4TB/EU_SCORES_project/WRFV4.4/EU_SCORES/New_runs/Netherlands_coast/Postprocessed/variablewise_files/wpd_80.nc'

In [30]:
import os
if os.path.exists(target_file):
    os.remove(target_file)

start = time.time()
wpd.to_netcdf(target_file)
end = time.time()
print(f'Wtiting {target_file} takes {end-start}s')

Wtiting /media/harish/SSD_4TB/EU_SCORES_project/WRFV4.4/EU_SCORES/New_runs/Netherlands_coast/Postprocessed/variablewise_files/wpd_80.nc takes 214.28972721099854s


In [31]:
del wpd